In [1]:
!pip install accelerate==0.25.0 bitsandbytes==0.41.1 datasets==2.14.6 peft==0.6.2 transformers==4.36.2 torch==2.1.0 einops==0.4.1 --quiet  # Phi needs this one

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,

In [4]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


In [5]:
# Load model
modelpath = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(
    modelpath,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    ),
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [101]:
# model.cuda()

In [6]:
from transformers import AutoTokenizer

# fast tokenizer sometimes ignores the added tokens
tokenizer = AutoTokenizer.from_pretrained(modelpath, use_fast=False)    

# add special tokens for ChatML formatting and a pad token
tokenizer.add_tokens(["<|im_start|>", "<PAD>"])
tokenizer.pad_token = "<PAD>"
tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))

# resize model embeddings
model.resize_token_embeddings(
    new_num_tokens=len(tokenizer),
    pad_to_multiple_of=64)   # phi2 default is 64, see configuration_phi.py
model.config.eos_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

In [5]:

# resize model embeddings
model.resize_token_embeddings(
    new_num_tokens=len(tokenizer),
    pad_to_multiple_of=64)   # phi2 default is 64, see configuration_phi.py
model.config.eos_token_id = tokenizer.eos_token_id

In [6]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Configure LoRA settings for fine-tuning
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules = ["q_proj",
#     "up_proj",
#     "o_proj",
#     "k_proj",
    "Wqkv",
    "out_proj"],
#     "down_proj",
#     "gate_proj",
#     "v_proj"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save = ["lm_head", "embed_tokens"],   # because we added new tokens
    task_type="CAUSAL_LM"
)
# lora_config.target_modules=['Wqkv','out_proj']
# Add adapters to model
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing = False)
model = get_peft_model(model, lora_config)
model.config.use_cache = False

In [7]:
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing = False)
model = get_peft_model(model, lora_config)
model.config.use_cache = False

In [106]:
model.cuda()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): PhiForCausalLM(
          (model): PhiModel(
            (embed_tokens): ModulesToSaveWrapper(
              (original_module): Embedding(51200, 2560)
              (modules_to_save): ModuleDict(
                (default): Embedding(51200, 2560)
              )
            )
            (embed_dropout): Dropout(p=0.0, inplace=False)
            (layers): ModuleList(
              (0-31): 32 x PhiDecoderLayer(
                (self_attn): PhiAttention(
                  (q_proj): Linear4bit(
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2560, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_fea

In [8]:
from datasets import load_dataset

# load the dataset created in Part 1
dataset = load_dataset("g-ronimo/riddles_evolved")

# split into training (90%) and test set (10%)
dataset = dataset["train"].train_test_split(test_size=0.1)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1682 [00:00<?, ? examples/s]

In [9]:
import os
from functools import partial

# ChatML format
templates = [
    "<|im_start|>assistant\n{msg}<|im_end|>",      # message by assistant
    "<|im_start|>user\n{msg}<|im_end|>"           # message by user
]


In [10]:

# This special index is used to ignore certain tokens during loss calculation.
IGNORE_INDEX = -100

def tokenize(input, max_length):
    input_ids, attention_mask, labels = [], [], []

    # Iterate over each message in the dataset
    for i, msg in enumerate(input["messages"]):

        # Check if the message is from human (user) or assistant, apply ChatML template
        isHuman = i%2==0
        msg_chatml = templates[isHuman].format(msg=msg)

        # tokenize all, truncate later
        msg_tokenized = tokenizer(
          msg_chatml, 
          truncation=False, 
          add_special_tokens=False)

        # Copy tokens and attention mask without changes
        input_ids += msg_tokenized["input_ids"]
        attention_mask += msg_tokenized["attention_mask"]

        # Adapt labels for loss calculation: if user->IGNORE_INDEX, if assistant->input_ids  (=ignore human messages, calculate loss only for assistant messages since these are the reponses we want to learn)
        labels += [IGNORE_INDEX]*len(msg_tokenized["input_ids"]) if isHuman else msg_tokenized["input_ids"]

    # truncate to max. length
    return {
        "input_ids": input_ids[:max_length], 
        "attention_mask": attention_mask[:max_length],
        "labels": labels[:max_length],
    }


In [11]:

dataset_tokenized = dataset.map(
    # cut samples at 1024 tokens
    # enough for the riddles dataset (max. length 1000 tokens)
    # has to be adapted for other datasets, higher=more VRAM needed
    partial(tokenize, max_length=1024), 
    batched = False,
    num_proc = os.cpu_count(),    # multithreaded
    remove_columns = dataset["train"].column_names  # Remove original columns, no longer needed
)

Map (num_proc=4):   0%|          | 0/1513 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/169 [00:00<?, ? examples/s]

In [14]:
def collate(elements):

    # Extract input_ids from each element and find the maximum length among them
    tokens = [e["input_ids"] for e in elements]
    tokens_maxlen = max([len(t) for t in tokens])

    for i, sample in enumerate(elements):
        input_ids = sample["input_ids"]
        labels = sample["labels"]
        attention_mask = sample["attention_mask"]

        # Calculate the padding length required to match the maximum token length
        pad_len = tokens_maxlen-len(input_ids)

        # Pad 'input_ids' with the pad token ID, 'labels' with IGNORE_INDEX, and 'attention_mask' with 0
        input_ids.extend( pad_len * [tokenizer.pad_token_id] )
        labels.extend( pad_len * [IGNORE_INDEX] )
        attention_mask.extend( pad_len * [0] )

    # create and return batch with all the data in elements
    batch={
        "input_ids": torch.tensor( [e["input_ids"] for e in elements] ),
        "labels": torch.tensor( [e["labels"] for e in elements] ),
        "attention_mask": torch.tensor( [e["attention_mask"] for e in elements] ),
    }
    return batch

In [15]:
from transformers import TrainingArguments, Trainer

bs=1         # batch size
ga_steps=16  # gradient acc. steps
epochs=20
lr=0.00002


In [16]:

steps_per_epoch=len(dataset_tokenized["train"])//(bs*ga_steps)

args = TrainingArguments(
    output_dir="out",
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    logging_steps=1,
    eval_steps=steps_per_epoch//2,      # eval twice per epoch
    save_steps=steps_per_epoch,         # save once per epoch
    gradient_accumulation_steps=ga_steps,
    num_train_epochs=epochs,
    lr_scheduler_type="constant",
    optim="paged_adamw_32bit",      # val_loss will go NaN with paged_adamw_8bit
    learning_rate=lr,
    group_by_length=False,
    bf16=False,
    ddp_find_unused_parameters=False,
)


In [17]:

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=collate,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_CUDA_mm)